In [1]:
from os import listdir
import jsonlines
import itertools
import nltk
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from tqdm import trange
from tqdm import tqdm

#nltk.download()
tknzr = nltk.TweetTokenizer()

In [2]:
path = listdir('c')
#path = ['AA', 'AD', 'AE', 'AF', 'AJ', 'AK', 'AQ', 'AU', 'BC', 'BD', 'BE', 'BH', 'BI', 'BJ', 'BK', 'BP', 'BW']
path = sorted(path)
print(path)

data = []
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AB', 'AD', 'AE', 'AF', 'AI', 'AJ', 'AK', 'AQ', 'AR', 'AT', 'AU', 'AV', 'AX', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BH', 'BI', 'BJ', 'BK', 'BO', 'BP', 'BR', 'BT', 'BU', 'BV', 'BW', 'BX']


### Подготваливаем заголовки

In [3]:
sw_ru = nltk.corpus.stopwords.words('russian')
morph = pm.MorphAnalyzer()



titles = [d['title'] for d in data]
stems = []
for i in trange(len(titles)):
    tokens = tknzr.tokenize(text=titles[i])
    tokens = [t.lower() for t in tokens if t.isalnum()]
    tokens = [morph.parse(t)[0].normal_form for t in tokens]
    tokens = [t for t in tokens if t not in sw_ru]
    titles[i] = ' '.join(tokens)
    stems.extend(tokens)
    
stems = sorted(set(stems))
print("Done")
print(stems[:100])

100%|████████████████████████████████████████████████████████████████████████| 739695/739695 [11:42<00:00, 1053.33it/s]


Done
['', '0', '00', '000', '00001', '000593', '001', '001223', '001a', '002', '003', '004', '00447', '0048', '005', '006', '007', '008', '0080', '0083', '009', '01', '010', '01011001', '011', '0110', '0110р', '012', '0120', '013', '014', '0145к', '0146', '015', '0150', '016', '0162', '017', '018', '0183', '019', '01м', '02', '020', '021', '022', '0220', '023', '0237', '0239', '024', '025', '02510', '026', '027', '02м', '03', '031', '032', '034', '0341l', '037', '03м', '04', '040', '0401', '040a', '041', '0416', '0426', '043', '049', '04983', '04b', '04м', '05', '05026', '0511', '0515', '051с', '052', '052c', '0535', '054', '056', '057', '0576', '058', '059', '05а', '05к', '05м', '05н', '06', '060', '060218', '0641', '0647', '067', '068']


In [4]:
#titles_set = sorted(set(titles))
titles_set = {}
for i in trange(len(data)):
    if titles_set.get(titles[i]) is not None:
        titles_set[titles[i]].append(data[i]['id'])
    else:
        titles_set[titles[i]] = [data[i]['id']]

100%|██████████████████████████████████████████████████████████████████████| 739695/739695 [00:03<00:00, 230575.18it/s]


## Подготавливаем категории

In [5]:
categories = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories))))

categories_dict = {c: 0 for c in categories}
for d in tqdm(data):
    for c in d['categories']:
        if categories_dict.get(c) is not None:
            categories_dict[c] += 1
            
ok = [c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000]
len(ok)

100%|██████████████████████████████████████████████████████████████████████| 739695/739695 [00:02<00:00, 301358.94it/s]


73556

In [6]:
#categories_dict = {c: [d['id'] for d in data if c in d['categories']] for c in ok}
categories_dict = {}
for c in tqdm(ok):
    categories_dict[c] = [d['id'] for d in data if c in d['categories']]

print("Done")

100%|██████████████████████████████████████████████████████████████████████████| 73556/73556 [3:47:26<00:00,  5.39it/s]


Done


In [39]:
list(categories_dict.items())[:4]

[('Преподаватели Московского государственного горного университета',
  ['171385',
   '638035',
   '2096671',
   '3198194',
   '3348239',
   '5290898',
   '5328232',
   '5861092',
   '6391854',
   '7005759',
   '6872794',
   '6872881',
   '6875907',
   '7006928']),
 ('Мебельные изделия',
  ['135845',
   '138031',
   '191202',
   '178660',
   '544212',
   '1874949',
   '3205038',
   '3658749',
   '3603807',
   '3589601',
   '6965822',
   '6838376']),
 ('Мультфильмы Владимира Сутеева',
  ['139936',
   '3297602',
   '4209483',
   '4549050',
   '5922414',
   '6698120',
   '6698129',
   '6826322',
   '7076041']),
 ('Византийские императоры VIII века',
  ['156270',
   '156784',
   '190367',
   '176070',
   '390454',
   '384614',
   '384944',
   '2446773'])]

## Вычисляем словарь категорий (слишком долго)

In [11]:
categories_vocab = {}

for c in tqdm(categories_dict.items()):
        
    '''vocab = set()
    for id in c[1]:
        titles = [key for key, value in titles_set.items() if id in value]
        for t in titles:
            vocab.update(re.split(r' ', t))
    categories_vocab[c[0]] = vocab
    '''
    vocab = set()
    titles = []
    for id in c[1]:
        titles.append(' '.join([key for key, value in titles_set.items() if id in value]))
    vocab = set(' '.join(titles).split())
    categories_vocab[c[0]] = vocab

  0%|                                                                            | 9/73556 [01:22<187:35:24,  9.18s/it]

KeyboardInterrupt: 

## Подготавливаем тексты _(как оказалось, это надо делать только для рассматриваемого документа)_

In [6]:
#texts = [d['text'] for d in data]
words = []

#CountVectorizer на таких объёмах не работал
with jsonlines.open('texts.jl', 'w') as output:
    for i, d in enumerate(data):
        if i % 1111 == 0:
            print(i, end=' ')

        text = re.sub(r'\n+', '. ', d['text'])
        text = re.sub(r'\xa0', ' ', text)
        tokens = tknzr.tokenize(text=text)
        tokens = [t.lower() for t in tokens if t.isalnum()]
        text = ' '.join(tokens)
        output.write({'text': text, 'id': d['id']})
        words.extend(sorted(set([t for t in tokens if t.isalpha()])))

print("Done")
words = sorted(set(words))
words[:1000]

0 1111 2222 3333 4444 5555 6666 7777 8888 9999 11110 12221 13332 14443 15554 16665 17776 18887 19998 21109 22220 23331 24442 25553 26664 27775 28886 29997 31108 32219 33330 34441 35552 36663 37774 38885 39996 41107 42218 43329 44440 45551 46662 47773 48884 49995 51106 52217 53328 54439 55550 56661 57772 58883 59994 61105 62216 63327 64438 65549 66660 67771 68882 69993 71104 72215 73326 74437 75548 76659 77770 78881 79992 81103 82214 83325 84436 85547 86658 87769 88880 89991 91102 92213 93324 94435 95546 96657 97768 98879 99990 101101 102212 103323 104434 105545 106656 107767 108878 109989 111100 112211 113322 114433 115544 116655 117766 118877 119988 121099 122210 123321 124432 125543 126654 127765 128876 129987 131098 132209 133320 134431 135542 136653 137764 138875 139986 141097 142208 143319 144430 145541 146652 147763 148874 149985 151096 152207 153318 154429 155540 156651 157762 158873 159984 161095 162206 163317 164428 165539 166650 167761 168872 169983 171094 172205 173316 17442

['a',
 'aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaa',
 'aaaaabbbbbabbbaabbababbaaababaab',
 'aaaaaух',
 'aaaab',
 'aaaabab',
 'aaaag',
 'aaaattga',
 'aaaaх',
 'aaaba',
 'aaabb',
 'aaabbbcccdddeeeffff',
 'aaabcccb',
 'aaabenraa',
 'aaae',
 'aaah',
 'aaai',
 'aaaja',
 'aaams',
 'aaargh',
 'aaarrg',
 'aaas',
 'aaasp',
 'aaass',
 'aaata',
 'aaav',
 'aaazoth',
 'aab',
 'aaba',
 'aabakken',
 'aabar',
 'aabb',
 'aabba',
 'aabbcc',
 'aabccb',
 'aabcccb',
 'aabccd',
 'aabcda',
 'aabenbaringen',
 'aaberg',
 'aac',
 'aaca',
 'aacademia',
 'aacayipsin',
 'aacaz',
 'aacc',
 'aacce',
 'aace',
 'aacgain',
 'aachen',
 'aachener',
 'aachi',
 'aachii',
 'aacimp',
 'aacm',
 'aaco',
 'aacplus',
 'aacs',
 'aacsa',
 'aacsb',
 'aacsbinternational',
 'aacta',
 'aacu',
 'aad',
 'aadhaar',
 'aadi',
 'aadress',
 'aads',
 'aadt',
 'aadvantage',
 'aae',
 'aaee',
 'aaen',
 'aaf',
 'aafc',
 'aafco',
 'aafdl',
 'aaff',
 'aafss',
 'aag',
 'aagaard',
 'aagad',
 'aagadu',
 'aagccta',
 'aagctta

In [7]:
# Велосипедим
def binary_search(a, x, lo=0, hi=None):
    hi = hi if hi is not None else len(a)
    pos = bisect_left(a, x, lo, hi)
    return (pos if pos != hi and a[pos] == x else -1)


normal_forms = {}
for i, w in enumerate(words):
    if i % 111111 == 0:
        print(i, end=' ')
    nf = morph.parse(w)[0].normal_form
    if binary_search(stems, nf) != -1:
        normal_forms[w] = nf

0 111111 222222 333333 444444 555555 666666 777777 888888 999999 1111110 1222221 1333332 1444443 1555554 1666665 1777776 1888887 1999998 2111109 2222220 2333331 2444442 2555553 2666664 2777775 2888886 2999997 

In [8]:
with jsonlines.open('texts.jl', 'r') as file:
    with jsonlines.open('normalized_texts.jl', 'w') as output:
        for i, f in enumerate(file):
            if i % 11111 == 0:
                print(i, end=' ')
            id = d['id']
            text = f['text']
            tokens = re.split(r' ', text)
            tokens = [normal_forms.get(t) for t in tokens if normal_forms.get(t) is not None]
            text = ' '.join(tokens)
            output.write({'text': text, 'id': id})


0 11111 22222 33333 44444 55555 66666 77777 88888 99999 111110 122221 133332 144443 155554 166665 177776 188887 199998 211109 222220 233331 244442 255553 266664 277775 288886 299997 311108 322219 333330 344441 355552 366663 377774 388885 399996 411107 422218 433329 444440 455551 466662 477773 488884 499995 511106 522217 533328 544439 555550 566661 577772 588883 599994 611105 622216 633327 644438 655549 666660 677771 688882 699993 711104 722215 733326 

## Начинаем анализировать документ

In [54]:
document = ''
i_d = 733327
with jsonlines.open('normalized_texts.jl', 'r') as input:
    for i, f in enumerate(input):
        if i == i_d:
            document = f['text']
            break

document

'директива ec директива ec известный директива батарейка аккумулятор отход батарейка аккумулятор нормативный акт который порядок использование утилизация батарейка аккумулятор документ принять сентябрь год совет европа сила сентябрь год принятие директива год отменить действие документ директива совет европа батарейка аккумулятор eэc март год директива еэс совет ес март год батарея аккумулятор определённый опасный вещество призвать сблизить законодательство государство член ес область однако несмотря идея цель дать директива полностью достигнуть июль год генеральный директорат коммуникация европейский комиссия разработать руководящий принцип регулировать будущий отношение сфера утилизация отход задача входить сокращение количество вредный вещество отход дальнейший резолюция совет ес январь год программа действие сообщество борьба загрязнение окружающий среда кадмий сделать акцент ограничение использование это химический элемент цель защита здоровье человек окружающий среда вредный влия

1. Взвешиваем слова в документe по формуле $ R_w = ft_w \log{\frac{N}{cf_w}} $

In [58]:
words = re.split(r' ', document)
words_set = set(words)
# Вычисляем tf_w
R_w = dict(Counter(words))
N = len(categories_dict)
# Вычисляем log(...)
for w in tqdm(words_set):
    cf_w = 0
    for c in categories_vocab:
        if w in c:
            cf_w += 1
    if cf_w > 0:
        R_w[w] = R_w[w] * log(N / cf_w)
    else:
        R_w[w] = 0
        

73556

2\. Взвешиваем заголовки по формуле $R_t = \sum_{w \rightarrow t} R_w \frac{1}{t_w} \frac{1}{a_t} \frac{S_t}{L_t}$

In [ ]:
R_t = dict()
title_vocab = dict(Counter(' '.join([sorted(titles_set)]).split()))
for title, ids in titles_set.items():
    title_words = title.split()
    supp_words = []
    flag = 1
    for w in title_words:
        if w not in words_set:
            flag = flag - 1
        else:
            supp_words.append(w)
    if flag < 0:
        continue
    sub_sum = 0
    # Вынесем 1 / a_t * S_t / L_t как mutual_mul
    mutual_mul = 1 / len(titles[title]) * len(supp_words) / len(title_words)
    for w in supp_words:
        sub_sum += R_w[w] / title_vocab[w]
        
    R_t[title] = mutual_mul * sub_sum

## Ниже черновик

In [ ]:
titles_set = sorted(set(titles))
print(len(titles))
print(titles_set[0:100])


739695
['', ' аистёс', '0', '0 2', '0 9', '0 a d', '0 е год до наш э', '0 сапфо', '0 январь', '00 династия', '00 крессмания', '001', '007 legends', '007 координата скайфолл', '007 спектр', '01 гельверция', '010 альбом the mad capsule markets', '01011001', '013 гордонмур', '034 акка', '04 испания', '05 гормуция', '05а 192 автодорога', '05а 214 автодорога', '05а 215 автодорога', '05а 608 автодорога', '05к 195 автодорога', '05к 442 автодорога', '05к 451 автодорога', '05н 100 автодорога', '05н 131 автодорога', '05н 263 автодорога', '060 аний', '07 вестур', '08а 10 автодорога', '09 лундия', '0verflow', '1', '1 1 1', '1 1 international', '1 1 медиа', '1 1 телеканал', '1 1 фильм', '1 2', '1 4', '1 9 8 2', '1 fille 4 types', '1 firebird', '1 gapa', '1 the collection', '1 wire', '1 август', '1 август пилотажный группа', '1 альбом хулио иглесиас', '1 апрель', '1 бром 3 хлор диметилгидантоин', '1 го отделение совхоз масловский', '1 го отделение совхоз новоусманский', '1 го ранг', '1 декабрь', '1 

In [7]:
title_to_article = nx.Graph()
for i, t in enumerate(titles):
    if i % 1111 == 0:
        print(i)
    title = next((ttl for ttl in titles_set if ttl == t), None)
    title_to_article.add_edge(title, int(data[i]['id']))


0
1111
2222
3333
4444
5555
6666
7777
8888
9999
11110
12221
13332
14443
15554
16665
17776
18887
19998
21109
22220


KeyboardInterrupt: 

In [ ]:
def binary_search(a, x, lo=0, hi=None):
    hi = hi if hi is not None else len(a)
    pos = bisect_left(a, x, lo, hi)
    return (pos if pos != hi and a[pos] == x else -1)



def process_article(i, t):
    if i % 101 == 0:
        print(i)
        
    tokens = tknzr.tokenize(text = t)
    tokens = [t.lower() for t in tokens if t.isalnum()]
    
    #вот тут скорее всего будет оч долго работать
    t = ''
    for tok in tokens:
        nf = morph.parse(tok)[0].normal_form
        if binary_search(stems, nf) != -1:
            t = t + nf + ' '
            
    t = t[:-1]

    
texts = Parallel(n_jobs=2)(delayed(process_article)(i, t) for i, t in enumerate(texts))

In [40]:
categories = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories))))
print("Categories - " + str(len(categories)))
print("Articles - " + str(len(data)))

categories

Categories - 222009
Articles - 382236


['',
 '.357 SIG firearms',
 '.40 S&W firearms',
 '.NET Framework',
 '.NET Framework|Compact Framework',
 '.NET Framework|Reflector',
 '.NET Framework|Remoting',
 '1',
 '1 августа',
 '1 апреля',
 '1 год',
 '1 год до н. э.',
 '1 декабря',
 '1 июля',
 '1 июня',
 '1 марта',
 '1 мая',
 '1 ноября',
 '1 октября',
 '1 сентября',
 '1 февраля',
 '1 января',
 '1+1 медиа',
 '1-й Белорусский фронт',
 '1-й Белорусский фронт|*',
 '1-й Дальневосточный фронт',
 '1-й МГМУ',
 '1-й Украинский фронт',
 '1-я Конная армия|*',
 '1-я Московская гимназия',
 '10 августа',
 '10 апреля',
 '10 год',
 '10 декабря',
 '10 июля',
 '10 июня',
 '10 марта',
 '10 мая',
 '10 ноября',
 '10 октября',
 '10 сентября',
 '10 февраля',
 '10 января',
 '10-я армия (Россия)',
 '10-я стрелковая дивизия внутренних войск НКВД (I)',
 '100 год',
 '100 магнитоальбомов советского рока',
 '1000 год',
 '1000-е годы до н. э.',
 '1001 год',
 '1002 год',
 '1004 год до н. э.',
 '1005 год',
 '101 год',
 '101 год до н. э.',
 '101 далматинец (анимац

In [124]:
vectorizer = CountVectorizer()
vectorizer.fit_transform([texts[0]])
vectorizer.vocabulary_

{'юкос': 1685,
 'оао': 883,
 'нк': 875,
 'кос': 659,
 'российская': 1309,
 'нефтяная': 861,
 'компания': 634,
 'существовавшая': 1477,
 '1993': 28,
 'по': 1044,
 '2007': 43,
 'год': 359,
 'августа': 138,
 'был': 242,
 'объявлен': 912,
 'банкротом': 208,
 'до': 430,
 '12': 11,
 'ноября': 881,
 'года': 360,
 'отношении': 986,
 'компании': 631,
 'осуществлялось': 962,
 'конкурсное': 647,
 'производство': 1211,
 'течение': 1503,
 'имущество': 574,
 'продавалось': 1201,
 'на': 769,
 'аукционах': 193,
 'целью': 1627,
 'покрытия': 1076,
 'долгов': 455,
 '21': 51,
 'была': 243,
 'ликвидирована': 699,
 'являлась': 1699,
 'одной': 927,
 'из': 561,
 'крупнейших': 682,
 'компаний': 632,
 'россии': 1308,
 'объёмам': 915,
 'реализации': 1277,
 'период': 1025,
 '1995': 29,
 '2005': 41,
 'неизменно': 835,
 'входила': 319,
 'число': 1652,
 '10': 7,
 'версии': 264,
 'журнала': 507,
 'эксперт': 1664,
 'лучший': 710,
 'результат': 1285,
 'место': 735,
 '2001': 37,
 '2003': 39,
 'годах': 361,
 'создание': 

In [53]:
nltk.word_tokenize('Я человек.')

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/maxfelmos/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [3]:
path = listdir('c')
i = 0
for p in path:
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            i += 1

print(i)

739450


In [12]:
d = {'a': 3, 'b': 3, 'c': 2}
[key for key, value in d.items() if value == 3]

['a', 'b']

In [15]:
for k, v in {'1': 1, '2': 2}.items():
    print(k)
    print(v)

1
1
2
2
